# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_data_path = 'output_data/cities.csv'

# Read the mouse data and the study results
cities_data = pd.read_csv(cities_data_path)

cities_data.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Port Hardy,50.6996,-127.4199,285.15,71,40,3.60,CA,Tue May 4 14:38:47 2021
1,1,Port Augusta,-32.5000,137.7667,285.15,66,3,5.14,AU,Tue May 4 14:38:48 2021
2,2,Hermanus,-34.4187,19.2345,292.04,82,18,1.79,ZA,Tue May 4 14:38:48 2021
3,3,Malé,4.1748,73.5089,302.15,84,75,1.03,MV,Tue May 4 14:38:49 2021
4,4,Registro,-24.4875,-47.8436,301.63,52,8,0.93,BR,Tue May 4 14:38:49 2021


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(g_key)

locations = cities_data[["Lat", "Lng"]].astype(float)

humidity_rate = cities_data["Humidity"].astype(float)

In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:


hotel_df = cities_data.loc[(cities_data["Max Temp"] > 158) & 
                           (cities_data["Wind Speed"] < 10) &
                                    (cities_data["Cloudiness"] == 0), :]
hotel_df.dropna(how ='any')

hotel_df.dtypes

Unnamed: 0      int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df["Hotel Name"] = hotel_df
hotel_df["Hotel Address"] = hotel_df 

hotel_df


ValueError: Wrong number of items passed 10, placement implies 1

In [ ]:
params = {
    "radius": 50000,
    "types": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        hotel_df.loc[index, "Hotel Address"] = name_address["results"][0]["vicinity"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add marker layer ontop of heat map
fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display figure
fig